# Processing Flow


## Install necessary modules


In [ ]:
%pip install uv
!uv pip install duckdb jupysql ipykernel sqlalchemy duckdb-engine pandas 


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Using Python 3.11.4 environment at: /Users/nivanrs/miniconda3
Resolved 57 packages in 2.67s                                        
⠙ Preparing packages... (0/10)                                                  
⠹ Preparing packages... (0/10)---     0 B/19.80 KiB                     
⠸ Preparing packages... (0/10)---     0 B/19.80 KiB                     
⠸ Preparing packages... (0/10)---     0 B/19.80 KiB                     
⠸ Preparing packages... (0/10)--- 14.88 KiB/19.80 KiB                   
⠸ Preparing packages... (0/10)/10)                                                  
⠸ Preparing packages... (0/10)---     0 B/513.73 KiB                    
posthog    ------------------------------     0 B/135.46 KiB
⠸ Preparing packages... (0/10)---     0 B/513.73 KiB                    
prettytable ------------------------------     0 B/33.07 KiB
posthog    --------------------

## Import Modules and Initiate DuckDB


In [33]:
import duckdb
from sqlalchemy import create_engine
import pandas as pd

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False

# Create DuckDB connection (saving)
%sql duckdb:///dev.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


## Closing Connection


In [34]:
%sql close duckdb:///dev.db

RuntimeError: If using snippets, you may pass the --with argument explicitly.
For more details please refer: https://jupysql.ploomber.io/en/latest/compose.html#with-argument


Original error message from DB driver:
(duckdb.duckdb.ParserException) Parser Error: syntax error at or near "close"
[SQL: close duckdb:///dev.db]
(Background on this error at: https://sqlalche.me/e/14/f405)



## Rollback


In [ ]:
%sql
ROLLBACK

## List Table


In [98]:
%sqlcmd tables

Name
dim_datetime
dim_tokens
dim_users
fact_trades
fact_transfers
stg_tokens
stg_trades
stg_transfers
stg_users


## data evaluation and data loading


### transfers


In [44]:
transfers = pd.read_csv("../../data/raw_p2p_transfers.csv")
transfers

,transfer_id,sender_id,receiver_id,token_id,amount,status,transfer_created_time,transfer_updated_time
0,P2P0001,U017,U005,SOL,136.340602,SUCCESS,2024-01-08 15:56:56,2024-01-08 15:56:56
1,P2P0002,U023,U008,USDT,2124.162930,SUCCESS,2024-01-28 5:52:29,2024-01-28 5:52:29
2,P2P0003,U014,U033,USDT,1315.962732,SUCCESS,2024-01-05 9:06:10,2024-01-05 9:06:10
3,P2P0004,U035,U001,USDT,1129.177555,SUCCESS,2024-01-29 15:33:26,2024-01-29 15:33:26
4,P2P0005,U032,U010,XRP,122.677632,SUCCESS,2024-01-06 16:29:03,2024-01-06 16:29:03
...,...,...,...,...,...,...,...,...
175,P2P0176,U010,U045,XRP,99.656284,SUCCESS,2024-06-11 23:01:38,2024-06-11 23:01:38
176,P2P0177,U007,U014,BTC,0.260988,FAILED,2024-06-05 1:45:39,2024-06-05 1:48:39
177,P2P0178,U005,U016,ETH,0.310714,FAILED,2024-06-15 14:28:24,2024-06-15 14:36:24
178,P2P0179,U006,U038,DOGE,29.513625,SUCCESS,2024-06-25 2:20:51,2024-06-25 2:20:51


consist of 8 cols, primary key on transfer_id, no missing but the time is stil not in date format


In [57]:
transfers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180 entries, 0 to 179
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   transfer_id            180 non-null    object 
 1   sender_id              180 non-null    object 
 2   receiver_id            180 non-null    object 
 3   token_id               180 non-null    object 
 4   amount                 180 non-null    float64
 5   status                 180 non-null    object 
 6   transfer_created_time  180 non-null    object 
 7   transfer_updated_time  180 non-null    object 
dtypes: float64(1), object(7)
memory usage: 11.4+ KB


#### Data loading


In [68]:
%%sql
DROP TABLE IF EXISTS stg_transfers;
CREATE TABLE IF NOT EXISTS stg_transfers (
    transfer_id TEXT,
    sender_id TEXT,
    receiver_id TEXT,
    token_id TEXT,
    amount DOUBLE,
    status TEXT,
    transfer_created_time TIMESTAMP,
    transfer_updated_time TIMESTAMP
);

INSERT INTO stg_transfers
SELECT * FROM read_csv_auto('../../data/raw_p2p_transfers.csv');


,Count
0,180


In [69]:
%%sql
SELECT * FROM stg_transfers;


,transfer_id,sender_id,receiver_id,token_id,amount,status,transfer_created_time,transfer_updated_time
0,P2P0001,U017,U005,SOL,136.340602,SUCCESS,2024-01-08 15:56:56,2024-01-08 15:56:56
1,P2P0002,U023,U008,USDT,2124.162930,SUCCESS,2024-01-28 05:52:29,2024-01-28 05:52:29
2,P2P0003,U014,U033,USDT,1315.962732,SUCCESS,2024-01-05 09:06:10,2024-01-05 09:06:10
3,P2P0004,U035,U001,USDT,1129.177555,SUCCESS,2024-01-29 15:33:26,2024-01-29 15:33:26
4,P2P0005,U032,U010,XRP,122.677632,SUCCESS,2024-01-06 16:29:03,2024-01-06 16:29:03
...,...,...,...,...,...,...,...,...
175,P2P0176,U010,U045,XRP,99.656284,SUCCESS,2024-06-11 23:01:38,2024-06-11 23:01:38
176,P2P0177,U007,U014,BTC,0.260988,FAILED,2024-06-05 01:45:39,2024-06-05 01:48:39
177,P2P0178,U005,U016,ETH,0.310714,FAILED,2024-06-15 14:28:24,2024-06-15 14:36:24
178,P2P0179,U006,U038,DOGE,29.513625,SUCCESS,2024-06-25 02:20:51,2024-06-25 02:20:51


### Tokens


In [54]:
tokens = pd.read_csv("../../data/raw_tokens.csv")
tokens

,token_id,token_name,category
0,ADA,Cardano,Layer-1
1,BNB,BNB,Layer-1
2,BTC,Bitcoin,Layer-1
3,DOGE,Dogecoin,Meme Coin
4,ETH,Ethereum,Layer-1
5,PEPE,Pepe,Meme Coin
6,SHIB,Shiba Inu,Meme Coin
7,SOL,Solana,Layer-1
8,USDT,Tether,Stablecoin
9,XRP,XRP,Layer-1


In [61]:
tokens.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   token_id    10 non-null     object
 1   token_name  10 non-null     object
 2   category    10 non-null     object
dtypes: object(3)
memory usage: 372.0+ bytes


#### Tokens data loading


In [63]:
%%sql
CREATE TABLE IF NOT EXISTS stg_tokens (
    token_id TEXT,
    token_name TEXT,
    category TEXT
);

INSERT INTO stg_tokens
SELECT * FROM read_csv_auto('../../data/raw_tokens.csv');


,Count
0,10


In [64]:
%%sql
SELECT * FROM stg_tokens;

,token_id,token_name,category
0,ADA,Cardano,Layer-1
1,BNB,BNB,Layer-1
2,BTC,Bitcoin,Layer-1
3,DOGE,Dogecoin,Meme Coin
4,ETH,Ethereum,Layer-1
5,PEPE,Pepe,Meme Coin
6,SHIB,Shiba Inu,Meme Coin
7,SOL,Solana,Layer-1
8,USDT,Tether,Stablecoin
9,XRP,XRP,Layer-1


### Trades


In [65]:
trades = pd.read_csv("../../data/raw_trades.csv")
trades

,trade_id,user_id,token_id,side,price_usd,quantity,status,trade_created_time,trade_updated_time
0,TR0297,U050,PEPE,SELL,0.000008,4.298629e+06,FILLED,2024-06-06 4:41:51,2024-06-06 4:41:51
1,TR0150,U008,XRP,BUY,0.573315,9.573114e+06,FILLED,2024-03-04 23:43:35,2024-03-04 23:43:35
2,TR0044,U048,ETH,SELL,3959.737804,4.387170e-02,FILLED,2024-01-04 12:17:43,2024-01-04 12:17:43
3,TR0234,U031,PEPE,SELL,0.000005,2.666318e+06,FILLED,2024-04-01 21:46:15,2024-04-01 21:46:15
4,TR0199,U023,XRP,SELL,0.642871,9.557742e+06,FILLED,2024-04-22 8:58:09,2024-04-22 8:58:09
...,...,...,...,...,...,...,...,...,...
298,TR0218,U015,SOL,BUY,170.890671,1.071587e+01,FILLED,2024-04-28 3:49:45,2024-04-28 3:49:45
299,TR0133,U006,BNB,SELL,593.594773,2.639536e+01,FILLED,2024-02-09 4:11:05,2024-02-09 4:11:05
300,TR0104,U037,XRP,BUY,0.540747,3.364109e+06,FILLED,2024-02-25 14:59:45,2024-02-25 14:59:45
301,TR0259,U032,PEPE,BUY,0.000014,3.493596e+08,FILLED,2024-05-13 7:48:39,2024-05-13 7:48:39


In [66]:
trades.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   trade_id            303 non-null    object 
 1   user_id             303 non-null    object 
 2   token_id            303 non-null    object 
 3   side                303 non-null    object 
 4   price_usd           303 non-null    float64
 5   quantity            303 non-null    float64
 6   status              303 non-null    object 
 7   trade_created_time  303 non-null    object 
 8   trade_updated_time  303 non-null    object 
dtypes: float64(2), object(7)
memory usage: 21.4+ KB


In [67]:
trades["trade_id"].value_counts()[lambda x: x > 1]

trade_id
TR0182    2
TR0072    2
TR0239    2
Name: count, dtype: int64

In [75]:
trades_deduped = trades.drop_duplicates(subset=["trade_id"])
trades_deduped

,trade_id,user_id,token_id,side,price_usd,quantity,status,trade_created_time,trade_updated_time
0,TR0297,U050,PEPE,SELL,0.000008,4.298629e+06,FILLED,2024-06-06 4:41:51,2024-06-06 4:41:51
1,TR0150,U008,XRP,BUY,0.573315,9.573114e+06,FILLED,2024-03-04 23:43:35,2024-03-04 23:43:35
2,TR0044,U048,ETH,SELL,3959.737804,4.387170e-02,FILLED,2024-01-04 12:17:43,2024-01-04 12:17:43
3,TR0234,U031,PEPE,SELL,0.000005,2.666318e+06,FILLED,2024-04-01 21:46:15,2024-04-01 21:46:15
4,TR0199,U023,XRP,SELL,0.642871,9.557742e+06,FILLED,2024-04-22 8:58:09,2024-04-22 8:58:09
...,...,...,...,...,...,...,...,...,...
298,TR0218,U015,SOL,BUY,170.890671,1.071587e+01,FILLED,2024-04-28 3:49:45,2024-04-28 3:49:45
299,TR0133,U006,BNB,SELL,593.594773,2.639536e+01,FILLED,2024-02-09 4:11:05,2024-02-09 4:11:05
300,TR0104,U037,XRP,BUY,0.540747,3.364109e+06,FILLED,2024-02-25 14:59:45,2024-02-25 14:59:45
301,TR0259,U032,PEPE,BUY,0.000014,3.493596e+08,FILLED,2024-05-13 7:48:39,2024-05-13 7:48:39


In [73]:
%%sql
CREATE TABLE IF NOT EXISTS stg_trades (
    trade_id TEXT PRIMARY KEY,
    user_id TEXT,
    token_id TEXT,
    side TEXT,
    price_usd DOUBLE,
    quantity DOUBLE,
    status TEXT,
    trade_created_time TIMESTAMP,
    trade_updated_time TIMESTAMP
);

,Count


In [83]:
%%sql
INSERT INTO stg_trades
SELECT trade_id,
       user_id,
       token_id,
       side,
       price_usd,
       quantity,
       status,
       trade_created_time,
       trade_updated_time
FROM (
  SELECT *,
         ROW_NUMBER() OVER (PARTITION BY trade_id ORDER BY trade_updated_time DESC) AS rn
  FROM read_csv_auto('../../data/raw_trades.csv')
)
WHERE rn = 1;


,Count
0,300


,Count
0,50


### Users


In [3]:
users = pd.read_csv("../../data/raw_users.csv")
users

,user_id,region,signup_date
0,U001,Jakarta,2023-05-10
1,U002,Medan,2023-08-09
2,U003,Medan,2023-04-17
3,U004,Surabaya,2023-04-15
4,U005,Makassar,2023-09-20
5,U006,Bandung,2023-04-17
6,U007,Yogyakarta,2023-03-23
7,U008,Medan,2023-09-20
8,U009,Makassar,2023-08-15
9,U010,Yogyakarta,2023-03-13


In [84]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      50 non-null     object
 1   region       50 non-null     object
 2   signup_date  50 non-null     object
dtypes: object(3)
memory usage: 1.3+ KB


In [86]:
%%sql
DROP TABLE IF EXISTS stg_users;
CREATE TABLE IF NOT EXISTS stg_users (
    user_id TEXT PRIMARY KEY,
    region TEXT,
    signup_date DATE
);

INSERT INTO stg_users
SELECT * FROM read_csv_auto('../../data/raw_users.csv');

,Count
0,50


## Fact and Dim Creation


### Dim table Creation


In [116]:
%%sql
DROP TABLE IF EXISTS dim_users;
CREATE TABLE dim_users AS
SELECT DISTINCT
    user_id,
    region,
    signup_date
FROM stg_users;

DROP TABLE IF EXISTS dim_tokens;
CREATE TABLE dim_tokens AS
SELECT DISTINCT
    token_id,
    token_name,
    category
FROM stg_tokens;

DROP TABLE IF EXISTS dim_datetime;
CREATE TABLE dim_datetime AS
SELECT DISTINCT
    DATE_TRUNC('day', trade_created_time) AS date_day,
    DATE_PART('year', trade_created_time) AS year,
    DATE_PART('month', trade_created_time) AS month
FROM stg_trades;


,Count
0,142


### Fact Table Creation


In [125]:
%%sql
DROP TABLE IF EXISTS fact_trades;
CREATE TABLE fact_trades AS
SELECT
    t.trade_id,
    t.user_id,
    t.token_id,
    t.side,
    t.price_usd,
    t.quantity,
    t.price_usd * t.quantity AS trade_value_usd,
    t.trade_created_time AS trade_date
FROM stg_trades t
WHERE t.status = 'FILLED';

DROP TABLE IF EXISTS fact_transfers;
CREATE TABLE fact_transfers AS
SELECT
    transfer_id,
    sender_id,
    receiver_id,
    token_id,
    amount,
    transfer_created_time AS transfer_date
FROM stg_transfers
WHERE status = 'SUCCESS';

,Count
0,164
